In [1]:
import pandas as pd
import spacy

from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin

In [2]:
df = pd.read_csv('../../../../raw_data/data_prep.csv')

In [3]:
train, dev = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
train.shape, dev.shape

((960, 8), (240, 8))

In [3]:
df['rating'].unique()

array(['A', 'BB', 'AAA', 'BBB', 'AA+', 'BB+', 'BB-', 'A-', 'A+', 'B',
       'AA-', 'BBB+', 'BBB-', 'B-', 'AA', 'B+', 'C'], dtype=object)

In [5]:
categories = ['B', 'A', 'A-', 'A+', 'B-', 'B+']

In [6]:
dev = dev.query('rating in @categories')

In [7]:
train = train.query('rating in @categories')

In [8]:
train.shape, dev.shape

((102, 8), (20, 8))

In [9]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [10]:
nlp = spacy.load('ru_core_news_lg')

In [11]:
def create_docbin(data):
    db = DocBin()
    global categories
    for i in range(data.shape[0]):    
        doc = nlp.make_doc(str(data["tokenized_str"][i]))
        doc.cats = {category: 0 for category in categories}
        doc.cats[data["rating"][i]] = 1
        db.add(doc)
    return db

In [12]:
create_docbin(train).to_disk("train.spacy")
create_docbin(dev).to_disk("dev.spacy")

In [13]:
! python -m spacy train config.cfg --output ./subsym_model --paths.train train.spacy --paths.dev dev.spacy

ℹ Saving to output directory: subsym_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       45.95    0.46
  1     200         18.05       45.95    0.46
  3     400          3.70       45.95    0.46
  5     600          0.00       45.95    0.46
  7     800          0.00       45.95    0.46
  9    1000          0.00       45.95    0.46
 11    1200          0.00       45.95    0.46
 13    1400          0.00       45.95    0.46
 15    1600          0.00       45.95    0.46
✔ Saved pipeline to output directory
subsym_model\model-last
